# Get 136 raw features

In this script we extracts 136 raw features from datafiles we generated in 8_processing.ipynb.

After this step, we have the following output files:
- DB_merged_Xhrs.npy: matrices of time series of each admission. For each admission, there is a matrix containing its records in given time period. Each row of the matrix is like this: [feature 0, …, feature n, number of seconds in [icu intime, current time ], admission_id].
- ICD9-Xhrs.npy: matrices of ICD9 codes of each admission. For each admission, there is a matrix containing its ICD9 codes. Each line of the matrix is like this: [admission_id, icd9 original code, icd9 parsed code, icd9 subcat number]
- AGE_LOS_MORTALITY_Xhrs.npy: matrices of the result of AGE_LOS_MORTALITY function for each admission. Here we just keep it for compatibility.
- ADM_FEATURES_Xhrs.npy: features only related to admissions and not related to time, containing age, whether there is AIDS/hematologic malignancy/metastatic cancer and admission type.
- ADM_LABELS_Xhrs.npy: mortality labels of all admissions, containing hospital mortality, 1/2/3-day mortality, 30-day mortality and 1-year mortality.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil
from collections import OrderedDict

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

%matplotlib inline

In [2]:
RESDIR = '../../Data/admdata_99p'
SOURCEDIR = '../../Data/admdata_valid/'
LABELDIR = './admdata_timelabels/'
PROCESSED_DB_DIR = '../../Data/admdata_17f/processed_db/'
RAWDIR = os.path.join(RESDIR, 'raw')
for dname in [RESDIR, RAWDIR]:
    if not os.path.exists(dname):
        os.makedirs(dname)

In [3]:
# load the aid used for previous exps
tempdball = np.load(os.path.join(RESDIR, '../admdata_17f/24hrs/DB_merged_24hrs.npy'))
valid_aids = [x[0][-1] for x in tempdball]
print(len(valid_aids))
print(valid_aids[:10])

35637
[100001, 100006, 100007, 100009, 100010, 100011, 100012, 100016, 100021, 100024]


In [4]:
map_itemid_index = np.load('res/map_itemid_index.npy').tolist()

## Define the map between features and itemids

Since the number of features is too big, we use an AST(abstract syntax tree) structure to define the mapping between features and itemids and the manipulation on features.

We use a dictionary as the top sturcture, the key is the name of the feature and the value is marked as L.

We define L as follows:

```
L -> list of [ID, MANI]
ID -> a specific itemid
MANI -> tuple(name of manipulation, L)
```

With this definition, we can easily parse the map recursively.

We conduct different manipulation according to the end of name of manipulation, and we define following methods:
- min
- max
- sum
- mean
- f2c: convert farenheit to celcius and use mean when in confliction
- lb2kg: convert lb to kg and use mean when in confliction
- oz2kg: convert oz to kg and use mean when in confliction
- inches2cm: convert inch to cm and use mean when in confliction

Other methods can be added. However, all methods should accept a list and return a single value.

In [5]:
# readin the csv file recording itemids
feature_itemids = OrderedDict({})

# added features
import csv
with open('./config/99plusf.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter=',', quotechar='\"')
    next(csvreader)
    for row in csvreader:
        feature_name = ''.join(row[3:])
        itemids = row[2].split(',')[:-1]
        inmapitemids = []
        for itemid in itemids:
            try:
                itemid = int(itemid)
            except:
                pass
            if itemid in map_itemid_index.keys():
                inmapitemids.append(itemid)
        if len(inmapitemids) > 0:
            feature_itemids[feature_name] = inmapitemids
            
# features from previous 17 raw features
feature_itemids_17raw = OrderedDict([
    ['gcsverbal', [723, 223900]],
    ['gcsmotor', [454, 223901]],
    ['gcseyes', [184, 220739]],
    ['systolic_blood_pressure_abp_mean', [51, 442, 455, 6701, 220050, 220179]],
    ['heart_rate', [211, 220045]],
    ['body_temperature', [
        ('_f2c', [678, 223761]),
        676,
        223762
    ]],
    ['pao2', [50821]],
    ['fio2', [50816, 223835, 3420, 3422, 190]],
    ['urinary_output_sum', [40055,
43175,
40069,
40094,
40715,
40473,
40085,
40057,
40056,
40405,
40428,
40086,
40096,
40651,
226559,
226560,
226561,
226584,
226563,
226564,
226565,
226567,
226557,
226558,
227488,
227489]],
    ['serum_urea_nitrogen_level', [51006]],
    ['white_blood_cells_count_mean', [51300, 51301]],
    ['serum_bicarbonate_level_mean', [50882]],
    ['sodium_level_mean', [50824, 50983]],
    ['potassium_level_mean', [50822, 50971]],
    ['bilirubin_level', [50885]],
])

for key, value in feature_itemids_17raw.items():
    feature_itemids[key] = value
    
# add the ratio of inspiratory:expiratory
feature_itemids['ie_ratio_mean'] = [
    221,
    ('_div', [226873, 226871])
]

# add other features
extra_features = OrderedDict([
    ['diastolic_blood_pressure_mean', [8368, 8440, 8441, 8555, 220180, 220051]],
    ['arterial_pressure_mean', [456, 52, 6702, 443, 220052, 220181, 225312]],
    ['respiratory_rate', [618, 615, 220210, 224690]],
    ['spo2_peripheral', [646, 220277]],
    ['glucose', [807, 811, 1529, 3745, 3744, 225664, 220621, 226537]],
    ['weight', [
        762, 763, 3723, 3580, 226512,
        ('_lb2kg', [3581]),
        ('_oz2kg', [3582])
    ]],
    ['height', [
        ('_inches2cm', [920, 1394, 4187, 3486,]),
        3485, 4188, 226707
    ]],
    ['hgb', [50811, 51222]],
    ['platelet', [51265]],
    ['chloride', [50806, 50902]],
    ['creatinine', [50912]],
    ['norepinephrine', [30047, 30120, 221906]],
    ['epinephrine', [30044,30119,30309,221289]],
    ['phenylephrine', [30127,30128,221749]],
    ['vasopressin', [30051,222315]],
    ['dopamine', [30043,30307,221662]],
    ['isuprel', [30046,227692]],
    ['midazolam', [30124, 221668]],
    ['fentanyl', [30150, 30308, 30118, 30149, 221744, 225972, 225942]],
    ['propofol', [30131, 222168]],
    ['peep', [50819]],
    ['ph', [50820]],
])

for key, value in extra_features.items():
    feature_itemids[key] = value
        
print(feature_itemids)

OrderedDict([('Gastric Gastric Tube', [40051, 40067, 226571, 226573, 226575, 226576]), ('Stool Out Stool', [40054, 226579]), ('Urine Out Incontinent', [40085, 40094, 226627]), ('Ultrafiltrate Ultrafiltrate', [40286, 226583]), ('Fecal Bag', [40053, 226580, 227701]), ('Ostomy (output)', [40289, 46071, 46709, 226582]), ('Chest Tube #1', [40048, 40049, 40076, 42255, 226588]), ('Chest Tube #2', [40050, 40090, 41707, 42256, 226589]), ('Jackson Pratt #1', [40071, 226599]), ('OR EBL', [40064, 226626]), ('Pre-Admission', [40060, 226633]), ('TF Residual', [45083, 227510, 227511]), ('Albumin 5%', [30008, 30181, 220864]), ('Fresh Frozen Plasma', [30005, 220970]), ('Lorazepam (Ativan)', [30141, 221385]), ('Calcium Gluconate', [30023, 221456]), ('Midazolam (Versed)', [45520, 221668]), ('Phenylephrine', [30127, 30128, 221749]), ('Furosemide (Lasix)', [30123, 221794]), ('Hydralazine', [42646, 221828]), ('Norepinephrine', [30047, 30120, 221906]), ('Magnesium Sulfate', ['MAG2PM']), ('Nitroglycerin', [30

In [6]:
# from copy import deepcopy
# mapped_feature_itemids = deepcopy(feature_itemids)
mapped_feature_itemids = OrderedDict({})

def mapping_features(fids, mapped, map_itemid_index=map_itemid_index):
    for key, value in fids.items():
        temp = mapping_lists(value, map_itemid_index)
        if temp is not None:
            mapped[key] = temp

def mapping_lists(fidlist, map_itemid_index=map_itemid_index):
    if type(fidlist) == type([]):
        mapped = []
        for fid in fidlist:
            temp = mapping_lists(fid, map_itemid_index)
            if temp is not None:
                mapped.append(temp)
        if len(mapped) > 0:
            return mapped
        else:
            return None
    elif type(fidlist) == type(()):
        temp = mapping_lists(fidlist[1], map_itemid_index)
        if temp is not None:
            return (fidlist[0], temp)
        else:
            return None
    else:
        assert type(fidlist) == type('str') or type(fidlist) == type(0)
        try:
            res = map_itemid_index[fidlist][0]
            return res
        except:
            return None
        
mapping_features(feature_itemids, mapped_feature_itemids)
print(mapped_feature_itemids)

map_feature_colids = {}
t = 0
for key in mapped_feature_itemids.keys():
    map_feature_colids[key] = t
    t += 1
print(len(map_feature_colids))
print(map_feature_colids)

OrderedDict([('Gastric Gastric Tube', [4029, 4068, 3738, 3544, 4075, 3211]), ('Stool Out Stool', [3862, 4020]), ('Urine Out Incontinent', [3475, 3353, 4311]), ('Ultrafiltrate Ultrafiltrate', [4052, 3432]), ('Fecal Bag', [3813, 4271, 4181]), ('Ostomy (output)', [3581, 3834, 4164, 4121]), ('Chest Tube #1', [4043, 4045, 3956, 3655, 3618]), ('Chest Tube #2', [4132, 3524, 4261, 4169, 3820]), ('Jackson Pratt #1', [3744, 3625]), ('OR EBL', [4044, 4047]), ('Pre-Admission', [3396, 3843]), ('TF Residual', [3730, 3775, 3506]), ('Albumin 5%', [1150, 353, 1523]), ('Fresh Frozen Plasma', [2253, 412]), ('Lorazepam (Ativan)', [1778, 845]), ('Calcium Gluconate', [1305, 63]), ('Midazolam (Versed)', [3151, 2432]), ('Phenylephrine', [2661, 2719, 2129]), ('Furosemide (Lasix)', [2014, 2845]), ('Hydralazine', [1214, 2655]), ('Norepinephrine', [1751, 873, 2738]), ('Magnesium Sulfate', [22872]), ('Nitroglycerin', [637, 1817]), ('Insulin - Regular', [1676, 2216, 1147]), ('Morphine Sulfate', [2314, 709]), ('Pota

In [7]:
np.save(os.path.join('../../Data/admdata_99p/raw/', 'map_feature_colids.npy'), map_feature_colids)

In [8]:
# merge selected items to one item; the value is the mean of all values
def keep_nonneg(x):
    try:
        x = float(x)
        return x >= 0
    except:
        return True

# Here we parse the feature_itemids like a grammar tree with top-down method, thus we can directly define our manipulation in the dict
# dict stands for non-leaf nodes and non-dict stands for leaf nodes
def merge_items(timeseries, mergenode, new_column_id, keep_func=keep_nonneg):
    # record timestamps
    if type(mergenode) == type(()):
        merged = {}
        mergelist = mergenode[1]
        mergename = mergenode[0]
        
        if mergename.endswith('_div'):
            merge_func = lambda x: x[0] / x[1]
        elif mergename.endswith('_min'):
            merge_func = np.nanmin
        elif mergename.endswith('_max'):
            merge_func = np.nanmax
        elif mergename.endswith('_sum'):
            merge_func = np.nansum
        elif mergename.endswith('_mean'):
            merge_func = np.nanmean
        elif mergename.endswith('_f2c'):
            merge_func = lambda x: (np.nanmean(x) - 32) * 5.0 / 9.0
        elif mergename.endswith('_lb2kg'):
            merge_func = lambda x: np.nanmean(x) * 0.453592
        elif mergename.endswith('_oz2kg'):
            merge_func = lambda x: np.nanmean(x) * 0.0283495
        elif mergename.endswith('_inches2cm'):
            merge_func = lambda x: np.nanmean(x) * 2.54
        else:
            merge_func = np.nanmean
            
        for mergeitem in mergelist:
            subtree = merge_items(timeseries, mergeitem, new_column_id, keep_func=keep_func)
            for record in subtree:
                try:
                    merged[record[0]].append(record[2])
                except KeyError:
                    merged[record[0]] = [record[2]]
        for key in merged.keys():
            try:
                merged[key] = merge_func(merged[key])
            except:
                merged[key] = None
        return [(key, new_column_id, value) for key, value in merged.items()]
    else:
        assert type(mergenode) == type('string') or type(mergenode) == type(0)
        subtree = []
        for record in timeseries:
            i, j, value = record[0], record[1], record[2]
            if j == mergenode:
                if value is not None and keep_func(value):
                    subtree.append((i, new_column_id, value))
        return subtree

# map itemids to column numbers
# def get_column_set(itemids, map_itemid_index):
#     return set([map_itemid_index[itemid][0] for itemid in itemids])

demoseries = [
    [0, 0, 1],
    [0, 1, 3],
    [0, 2, 4]
]
demomerge = {
    'ratio': [
        0,
        ('pf_div', [1, 2])
    ]
}

for key, value in demomerge.items():
    print(merge_items(demoseries, (key, value), 2233))

[(0, 2233, 0.875)]


In [9]:
def extract_serial_features(aid, res):
    timeseries, general, icd9 = res['timeseries']['codes'], res['general'], res['icd9']
    D = res['timeseries']['features']
    assert general[0] == aid
    new_timeseries = []
    for featurename, featurelist in mapped_feature_itemids.items():
        temp = merge_items(timeseries, (featurename, featurelist), map_feature_colids[featurename])
        if temp is not None and len(temp) > 0:
            new_timeseries.append(temp)
            
    # get timestamps
    timecolid = len(map_feature_colids)
    valid_timestamps = list(itertools.chain(*[[t[0] for t in tt] for tt in new_timeseries]))
    max_timestamp = max(valid_timestamps)
    timestamps = [(t[0], timecolid, t[2]) for t in timeseries if t[0] <= max_timestamp and t[1] == D - 2]
    new_timeseries.append(timestamps)
    timelength = max([t[2] for t in timestamps]) - min([t[2] for t in timestamps])
    
    # aid
    aidcolid = timecolid + 1
    aids = [(t[0], aidcolid, t[2]) for t in timeseries if t[0] <= max_timestamp and t[1] == D - 1]
    new_timeseries.append(aids)
    
    res2 = list(itertools.chain(*new_timeseries))
    return {'timestep':max_timestamp + 1,'features':aidcolid+1,'codes':res2, 'timelength':timelength}

adm = np.load(os.path.join('admdata', 'adm-127294.npy')).tolist()
extract_serial_features(127294, adm)

{'codes': [(220, 0, 30.0),
  (1314, 0, 10.0),
  (135, 0, 300.0),
  (206, 0, 250.0),
  (250, 0, 250.0),
  (287, 0, 100.0),
  (293, 0, 5.0),
  (437, 0, 100.0),
  (479, 0, 200.0),
  (537, 0, 350.0),
  (561, 0, 130.0),
  (571, 0, 5.0),
  (572, 0, 30.0),
  (576, 0, 60.0),
  (583, 0, 70.0),
  (1097, 0, 250.0),
  (1117, 0, 100.0),
  (1125, 0, 0.0),
  (1477, 1, 120.0),
  (1507, 1, 80.0),
  (1833, 1, 1000.0),
  (1935, 1, 800.0),
  (2018, 1, 400.0),
  (585, 11, 40.0),
  (591, 11, 30.0),
  (605, 11, 30.0),
  (612, 11, 15.0),
  (622, 11, 5.0),
  (637, 11, 5.0),
  (648, 11, 5.0),
  (687, 11, 5.0),
  (694, 11, 8.0),
  (817, 11, 120.0),
  (835, 11, 60.0),
  (839, 11, 50.0),
  (850, 11, 5.0),
  (907, 11, 10.0),
  (925, 11, 10.0),
  (1011, 11, 5.0),
  (1272, 11, 120.0),
  (1279, 11, 70.0),
  (1299, 11, 50.0),
  (1326, 11, 5.0),
  (1396, 11, 5.0),
  (1406, 11, 5.0),
  (1432, 11, 38.0),
  (2018, 11, 15.0),
  (2036, 11, 0.0),
  (56, 13, 273.00001200000003),
  (58, 13, 281.99998799999997),
  (109, 13, 288.

In [ ]:
def extract_adm_features(processed_adm):
    # get from previous in database
    return processed_adm['adm_features']

def extract_adm_labels(aid):
    admlabel = np.load(os.path.join(LABELDIR, 'adm-%.6d.npy' % aid)).tolist()
    adm_labels = (
        admlabel['mor'],
        admlabel['mor24'],
        admlabel['mor48'],
        admlabel['mor72'],
        admlabel['mor30d'],
        admlabel['mor1y'],
    )
    return adm_labels


def extract_adm(aid):
    admres = np.load(os.path.join(SOURCEDIR, 'adm-{0}.npy'.format(str('%.6d' % aid)))).tolist()
    processed_admres = np.load(os.path.join(PROCESSED_DB_DIR, 'adm-{0}.npy'.format(str('%.6d' % aid)))).tolist()
    res = {
        'serial_features': extract_serial_features(aid, admres),
        'adm_features': extract_adm_features(processed_admres),
        'adm_labels': extract_adm_labels(aid)
    }
    np.save(os.path.join(RAWDIR, 'adm-{0}.npy'.format(str('%.6d' % aid))), res)
    print('finished {0}!'.format(aid))
    
extract_adm(164444)

finished 164444!


In [ ]:
p = Pool()
for aid in valid_aids:
    p.apply_async(extract_adm, args=(aid,))
p.close()
p.join()

finished 100001!
finished 100010!
finished 100006!
finished 100009!
finished 100024!
finished 100007!
finished 100033!
finished 100038!
finished 100028!
finished 100030!
finished 100034!
finished 100041!
finished 100040!
finished 100065!
finished 100046!
finished 100071!
finished 100016!
finished 100094!
finished 100078!
finished 100066!
finished 100075!
finished 100112!
finished 100050!
finished 100072!
finished 100012!
finished 100063!
finished 100113!
finished 100124!
finished 100021!
finished 100059!
finished 100118!
finished 100099!
finished 100087!
finished 100036!
finished 100114!
finished 100136!
finished 100126!
finished 100143!
finished 100142!
finished 100116!
finished 100147!
finished 100119!
finished 100161!
finished 100146!
finished 100037!
finished 100173!
finished 100031!
finished 100045!
finished 100053!
finished 100074!
finished 100168!
finished 100131!
finished 100160!
finished 100186!
finished 100224!
finished 100195!
finished 100232!
finished 100225!
finished 10019

finished 101324!
finished 101325!
finished 101348!
finished 101334!
finished 101333!
finished 101353!
finished 101114!
finished 101297!
finished 101373!
finished 101317!
finished 101374!
finished 101394!
finished 101360!
finished 101340!
finished 101371!
finished 101344!
finished 101381!
finished 101299!
finished 101215!
finished 101427!
finished 101263!
finished 101098!
finished 101424!
finished 101399!
finished 101408!
finished 101430!
finished 101057!
finished 101406!
finished 101379!
finished 101170!
finished 101404!
finished 101400!
finished 101352!
finished 101432!
finished 101453!
finished 101439!
finished 101117!
finished 101388!
finished 101428!
finished 101443!
finished 101463!
finished 101457!
finished 101435!
finished 101471!
finished 101496!
finished 101397!
finished 101516!
finished 101441!
finished 101501!
finished 101503!
finished 101504!
finished 101509!
finished 101524!
finished 101515!
finished 101460!
finished 101295!
finished 101517!
finished 101499!
finished 10144

finished 102696!
finished 102693!
finished 102515!
finished 102683!
finished 102610!
finished 102726!
finished 102657!
finished 102739!
finished 102716!
finished 102695!
finished 102768!
finished 102771!
finished 102724!
finished 102738!
finished 102694!
finished 102786!
finished 102725!
finished 102783!
finished 102722!
finished 102535!
finished 102575!
finished 102789!
finished 102791!
finished 102781!
finished 102754!
finished 102816!
finished 102801!
finished 102712!
finished 102664!
finished 102830!
finished 102825!
finished 102063!
finished 102821!
finished 102513!
finished 102772!
finished 102790!
finished 102808!
finished 102741!
finished 102841!
finished 102838!
finished 102860!
finished 102793!
finished 102852!
finished 102848!
finished 102872!
finished 102853!
finished 102867!
finished 102869!
finished 102645!
finished 102803!
finished 102797!
finished 102835!
finished 102862!
finished 102828!
finished 102874!
finished 102807!
finished 102857!
finished 102439!
finished 10288

finished 104004!
finished 104007!
finished 104003!
finished 104037!
finished 104036!
finished 104023!
finished 104009!
finished 103951!
finished 104032!
finished 104034!
finished 104047!
finished 103722!
finished 103999!
finished 104073!
finished 104021!
finished 104080!
finished 104025!
finished 104085!
finished 103971!
finished 104052!
finished 104030!
finished 104089!
finished 103741!
finished 104059!
finished 104100!
finished 104048!
finished 104083!
finished 104000!
finished 104040!
finished 104087!
finished 104101!
finished 103989!
finished 104095!
finished 104117!
finished 104091!
finished 103889!
finished 104113!
finished 104124!
finished 104042!
finished 103754!
finished 104130!
finished 104084!
finished 104093!
finished 104107!
finished 104114!
finished 104110!
finished 104125!
finished 104145!
finished 104160!
finished 104094!
finished 103947!
finished 104057!
finished 104158!
finished 103856!
finished 104154!
finished 104131!
finished 104161!
finished 104167!
finished 10419

finished 104680!
finished 105397!
finished 105392!
finished 105346!
finished 105385!
finished 105280!
finished 105404!
finished 105393!
finished 105401!
finished 105391!
finished 105400!
finished 104984!
finished 105357!
finished 105371!
finished 105437!
finished 105323!
finished 105449!
finished 105462!
finished 105270!
finished 105399!
finished 105446!
finished 105447!
finished 105434!
finished 105248!
finished 105442!
finished 105445!
finished 105418!
finished 105444!
finished 105452!
finished 105454!
finished 105466!
finished 105430!
finished 105406!
finished 105429!
finished 105433!
finished 105419!
finished 105279!
finished 105502!
finished 105484!
finished 104948!
finished 105457!
finished 105513!
finished 105458!
finished 105361!
finished 105504!
finished 105471!
finished 105381!
finished 105510!
finished 105331!
finished 105487!
finished 105518!
finished 105511!
finished 105516!
finished 105506!
finished 105515!
finished 105530!
finished 105551!
finished 105553!
finished 10550

...

finished 151949!
finished 153060!
finished 152179!
finished 153049!
finished 153020!
finished 153083!
finished 153077!
finished 153070!
finished 153092!
finished 153100!
finished 153064!
finished 153091!
finished 153106!
finished 153069!
finished 153121!
finished 153105!
finished 152943!
finished 153122!
finished 153103!
finished 152870!
finished 153119!
finished 153124!
finished 153107!
finished 153094!
finished 153098!
finished 153155!
finished 153126!
finished 153136!
finished 153111!
finished 152841!
finished 153129!
finished 152990!
finished 153141!
finished 152740!
finished 153164!
finished 153143!
finished 152675!
finished 153174!
finished 153172!
finished 153110!
finished 152446!
finished 153062!
finished 153185!
finished 153163!
finished 153191!
finished 153112!
finished 153193!
finished 153156!
finished 153200!
finished 153013!
finished 153225!
finished 153214!
finished 153219!
finished 153181!
finished 153212!
finished 153223!
finished 153226!
finished 153232!
finished 15317

finished 154163!
finished 154332!
finished 154424!
finished 154386!
finished 154427!
finished 154404!
finished 154434!
finished 154202!
finished 154417!
finished 154426!
finished 154411!
finished 154429!
finished 154394!
finished 154416!
finished 154402!
finished 154444!
finished 154406!
finished 154138!
finished 154452!
finished 154472!
finished 154455!
finished 154451!
finished 154469!
finished 154470!
finished 154478!
finished 154446!
finished 154438!
finished 154393!
finished 154475!
finished 154448!
finished 154430!
finished 154214!
finished 154218!
finished 154493!
finished 154486!
finished 154175!
finished 154435!
finished 154502!
finished 154490!
finished 154483!
finished 154509!
finished 154487!
finished 154496!
finished 154468!
finished 154439!
finished 154515!
finished 154442!
finished 154514!
finished 154523!
finished 154552!
finished 154542!
finished 154517!
finished 154545!
finished 154473!
finished 154506!
finished 154276!
finished 154554!
finished 154556!
finished 15451

finished 155784!
finished 155760!
finished 155761!
finished 155769!
finished 155834!
finished 155814!
finished 155805!
finished 155810!
finished 155849!
finished 155799!
finished 155843!
finished 155797!
finished 155854!
finished 155869!
finished 155871!
finished 155817!
finished 155798!
finished 155877!
finished 155866!
finished 155851!
finished 155874!
finished 155853!
finished 155807!
finished 155876!
finished 155875!
finished 155892!
finished 155812!
finished 155612!
finished 155904!
finished 155883!
finished 155893!
finished 155906!
finished 155806!
finished 155919!
finished 155715!
finished 155922!
finished 155576!
finished 155913!
finished 155917!
finished 155795!
finished 155940!
finished 155945!
finished 155880!
finished 155905!
finished 155928!
finished 155594!
finished 155956!
finished 155803!
finished 155962!
finished 155885!
finished 155950!
finished 155944!
finished 155961!
finished 155958!
finished 155605!
finished 155976!
finished 155988!
finished 155984!
finished 15598

finished 157106!
finished 157141!
finished 157196!
finished 155705!
finished 157203!
finished 157256!
finished 157239!
finished 157262!
finished 157294!
finished 157277!
finished 157229!
finished 156965!
finished 157284!
finished 157260!
finished 157159!
finished 157289!
finished 156837!
finished 157269!
finished 157213!
finished 157212!
finished 157211!
finished 157281!
finished 157295!
finished 157194!
finished 156903!
finished 157309!
finished 157322!
finished 157302!
finished 157332!
finished 156549!
finished 157271!
finished 157286!
finished 157053!
finished 157313!
finished 156847!
finished 157321!
finished 157318!
finished 157348!
finished 157357!
finished 157353!
finished 157362!
finished 157304!
finished 157263!
finished 157176!
finished 157344!
finished 157358!
finished 157156!
finished 157372!
finished 157361!
finished 157297!
finished 157396!
finished 157308!
finished 157403!
finished 157416!
finished 157400!
finished 157411!
finished 157398!
finished 157338!
finished 15740

finished 158614!
finished 158667!
finished 158678!
finished 158619!
finished 158389!
finished 158675!
finished 158670!
finished 158684!
finished 158665!
finished 158660!
finished 158635!
finished 158571!
finished 158691!
finished 158689!
finished 158677!
finished 158685!
finished 158708!
finished 158555!
finished 158676!
finished 158661!
finished 158392!
finished 158697!
finished 158561!
finished 158592!
finished 158646!
finished 158703!
finished 158742!
finished 158728!
finished 158741!
finished 158715!
finished 158744!
finished 158732!
finished 158688!
finished 158739!
finished 158754!
finished 158748!
finished 158724!
finished 158707!
finished 158596!
finished 158729!
finished 158769!
finished 158718!
finished 158711!
finished 158721!
finished 158001!
finished 158758!
finished 158749!
finished 158782!
finished 158791!
finished 158797!
finished 158809!
finished 158807!
finished 158768!
finished 158810!
finished 158824!
finished 158826!
finished 158811!
finished 158827!
finished 15870

## Select admissions with > xxhrs records

In this step we only keep admissions with record length > 24/48 hrs.

In [5]:
TARGETDIR = RESDIR
def extract_time_series(aidres):
    time_series = aidres['serial_features']
    tn = time_series['timestep']
    fn = time_series['features']
    codes = time_series['codes']
    series = [[None for ft in range(fn)] for tt in range(tn)]
    for tt, ft, value in codes:
        try:
            series[tt][ft] = value
        except:
            print(tt, ft, value, tn, fn)
    return series

def check_adm_hrs_pass(admres, hrs):
    return admres['serial_features']['timelength'] > hrs * 3600.0
    
def extract_data(aid, hrs):
    admres = np.load(os.path.join(RAWDIR, 'adm-%.6d.npy' % aid)).tolist()
    if check_adm_hrs_pass(admres, hrs):
        ori_admres = np.load(os.path.join(SOURCEDIR, 'adm-%.6d.npy' % aid)).tolist()
        return (
            extract_time_series(admres),
            ori_admres['icd9'],
            ori_admres['general'],
            admres['adm_features'],
            admres['adm_labels']
        )
    else:
        return None
        
def collect_admissions_with_more_than_hrs(hrs):
    processed_valid_aids = valid_aids
    
    HRDIR = os.path.join(TARGETDIR, '%dhrs_raw' % hrs)
    if not os.path.exists(HRDIR):
        os.makedirs(HRDIR)
        
    p = Pool()
    collec = [p.apply_async(extract_data, args=(aid, hrs)) for aid in processed_valid_aids]
    p.close()
    p.join()
    collec = [x.get() for x in collec]
    collec = [x for x in collec if x is not None]
    
    data_all = [r[0] for r in collec]
    label_icd9_all = [r[1] for r in collec]
#     label_mor_all = [r[2][:6] for r in collec]
    label_mor_all = [r[2] for r in collec]
    adm_features_all = [r[3] for r in collec]
    adm_labels_all = [r[4] for r in collec]
    
    np.save(os.path.join(HRDIR, 'DB_merged_%dhrs.npy' % hrs), data_all)
    np.save(os.path.join(HRDIR, 'ICD9-%dhrs.npy' % hrs), label_icd9_all)
    np.save(os.path.join(HRDIR, 'AGE_LOS_MORTALITY_%dhrs.npy' % hrs), label_mor_all)
    np.save(os.path.join(HRDIR, 'ADM_FEATURES_%dhrs.npy' % hrs), adm_features_all)
    np.save(os.path.join(HRDIR, 'ADM_LABELS_%dhrs.npy' % hrs), adm_labels_all)

# >= 24hrs
# collect_admissions_with_more_than_hrs(24)
collect_admissions_with_more_than_hrs(48)